In [1]:
DEVICE = 'cuda:2'
DATA_PATH = '/workspace/code/Akash/ImageNet'
BATCH_SIZE = 64

In [2]:
import os
import sys
import torch
from torchvision import transforms as transforms
from torchvision import datasets as datasets
sys.path.append("../../")
torch.cuda.set_device(int(DEVICE[-1]))

In [3]:
def build_imagenet_data(data_path: str = '', input_size: int = 224, batch_size: int = 64, workers: int = 4,
                        dist_sample: bool = False):

    traindir = os.path.join(data_path, 'train')
    valdir = os.path.join(data_path, 'val')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    #torchvision.set_image_backend('accimage')
    train_dataset = datasets.ImageFolder(
        traindir,
        transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
    val_dataset = datasets.ImageFolder(
        valdir,
        transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            normalize,
        ]))

    if dist_sample:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
        val_sampler = torch.utils.data.distributed.DistributedSampler(val_dataset)
    else:
        train_sampler = None
        val_sampler = None

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=(train_sampler is None),
        num_workers=workers, pin_memory=True, sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(
        val_dataset,batch_size=batch_size, shuffle=False,
        num_workers=workers, pin_memory=True, sampler=val_sampler)
    print('==> Using Imagenet Dataset')
    
    return train_loader, val_loader

In [4]:
trainloader, valloader = build_imagenet_data(data_path=DATA_PATH, batch_size=BATCH_SIZE)
dataloaders = {'train' : trainloader,'val' : valloader}

==> Using Imagenet Dataset


In [5]:
# import libraries
from trailmet.models import resnet
from trailmet.algorithms.quantize.lapq import LAPQ

In [6]:
# load model
cnn = resnet.get_resnet_model('resnet50', 1000, 224, pretrained=True)

In [7]:
# test model
from trailmet.algorithms.algorithms import BaseAlgorithm
BaseAlgorithm().test(model=cnn, dataloader=dataloaders['val'], device=torch.device(DEVICE))

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 754/754 [02:30<00:00,  5.01it/s, acc1=80.6, acc5=95.2]


(80.57272445112072, 95.23050974028813)

In [8]:
# quantize model
kwargs = {
    'W_BITS':6, 
    'A_BITS':8, 
    'ACT_QUANT':True,
    'CALIB_BATCHES':1024//BATCH_SIZE, 
    'MAX_ITER':1,
    'MAX_FEV':1,
    'VERBOSE':True,
    'PRINT_FREQ':20,
    'GPU_ID':int(DEVICE[-1]),
    'SEED':42
    }
qnn = LAPQ(cnn, dataloaders, **kwargs)
qnn.compress_model()

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 754/754 [03:40<00:00,  3.41it/s, acc1=10.9, acc5=25.5]


Quantization (W6A8) accuracy before LAPQ: (10.90434350132626, 25.542487169766616)


100%|████████████████████████████████████████████████████████████████████████████████████| 754/754 [03:43<00:00,  3.38it/s, acc1=11.8, acc5=25.7, loss=5.52]


==> p intr : 5.78
==> loss : 5.5192
==> acc@1 | acc@5 : 11.8182 | 25.7220
==> iteration: 0, minimum loss so far: 5.7969
==> iteration: 20, minimum loss so far: 5.5168
==> iteration: 40, minimum loss so far: 5.1465
==> iteration: 60, minimum loss so far: 4.8522
==> iteration: 80, minimum loss so far: 4.3918
==> iteration: 100, minimum loss so far: 3.9763
==> iteration: 120, minimum loss so far: 3.9236
==> iteration: 140, minimum loss so far: 3.6523
==> iteration: 160, minimum loss so far: 2.4104
==> iteration: 180, minimum loss so far: 2.4088
==> iteration: 200, minimum loss so far: 2.3857
==> iteration: 220, minimum loss so far: 2.2027
==> iteration: 240, minimum loss so far: 2.1963
==> iteration: 260, minimum loss so far: 2.0901
==> iteration: 280, minimum loss so far: 2.0862
==> iteration: 300, minimum loss so far: 2.0726
==> iteration: 320, minimum loss so far: 1.9923
==> iteration: 340, minimum loss so far: 1.9888
==> iteration: 360, minimum loss so far: 1.8058
==> iteration: 380, 

KeyboardInterrupt: 

In [9]:
from trailmet.algorithms.algorithms import BaseAlgorithm
BaseAlgorithm().test(model=qnn.model, dataloader=dataloaders['val'], device=torch.device(DEVICE))

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 754/754 [04:56<00:00,  2.54it/s, acc1=72.3, acc5=91.2]


(72.25548884343722, 91.2415486950457)